In [1]:
!git clone https://github.com/mehd-io/duckdb-playground-tutorial

Cloning into 'duckdb-playground-tutorial'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 52 (delta 16), reused 46 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 145.70 KiB | 921.00 KiB/s, done.
Resolving deltas: 100% (16/16), done.


In [2]:
# No DuckDB ? Difference between DuckDB Engine ?
!pip install --quiet jupysql
!pip install --quiet duckdb-engine==0.10.0
!pip install --quiet pandas

In [7]:
import duckdb
import pandas as pd
import getpass
import sqlalchemy

We configure jupysql to return data as a Pandas dataframe and have less verbose output

In [9]:
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 10)

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Connecting to DuckDB
Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [10]:
# use database url containing md: if you want to connect to motherduck
#database_url = f"duckdb:///md:sample_data?motherduck_token={motherduck_token}"
database_url = f"duckdb:///:memory:"
engine = sqlalchemy.create_engine(database_url)

In [11]:
%sql engine

## Querying DuckDB
Single line SQL queries can be run using `%sql` at the start of a line. Query results will be displayed as a Pandas DF. Note the SQL syntax highlighting!

In [12]:
%%sql
SELECT extension_name, installed, description
FROM duckdb_extensions();

,extension_name,installed,description
0,arrow,False,A zero-copy data integration between Apache Ar...
1,autocomplete,False,Adds support for autocomplete in the shell
...,...,...,...
21,tpch,True,Adds TPC-H data generation and query support
22,vss,False,Adds indexing support to accelerate Vector Sim...


In [13]:
## Gettting your connection ready
import duckdb
con = duckdb.connect()

## Play with some pandas DF and Python

In [5]:
df

,As of,Rank,Year to Date Rank,Last Week Rank,Title,Type,Netflix Exclusive,Netflix Release Date,Days In Top 10,Viewership Score
0,2020-04-01,1,1,1,"Tiger King: Murder, Mayhem …",TV Show,Yes,"Mar 20, 2020",9,90
1,2020-04-01,2,2,-,Ozark,TV Show,Yes,"Jul 21, 2017",5,45
2,2020-04-01,3,3,2,All American,TV Show,None,"Mar 28, 2019",9,76
3,2020-04-01,4,4,-,Blood Father,Movie,None,"Mar 26, 2020",5,30
4,2020-04-01,5,5,4,The Platform,Movie,Yes,"Mar 20, 2020",9,55
...,...,...,...,...,...,...,...,...,...,...
7095,2022-03-11,6,5,1,Worst Roommate Ever,TV Show,Yes,"Mar 1, 2022",10,81
7096,2022-03-11,7,7,2,Vikings: Valhalla,TV Show,Yes,"Feb 25, 2022",14,100
7097,2022-03-11,8,8,-,Shooter,Movie,None,"Aug 1, 2014",3,7
7098,2022-03-11,9,9,7,Shrek 2,Movie,None,"Mar 1, 2022",10,33


In [6]:
# query directly pandas dataframe!
con.sql("""SELECT Title, max("Days In Top 10") from df
where Type='Movie'
GROUP BY Title
ORDER BY max("Days In Top 10") desc
limit 5;""")

┌────────────────────────────────┬───────────────────────┐
│             Title              │ max("Days In Top 10") │
│            varchar             │         int64         │
├────────────────────────────────┼───────────────────────┤
│ The Mitchells vs. The Machines │                    31 │
│ Vivo                           │                    29 │
│ How the Grinch Stole Christmas │                    29 │
│ 365 Days                       │                    28 │
│ Despicable Me 2                │                    27 │
└────────────────────────────────┴───────────────────────┘

## Connect to MotherDuck

In [3]:
import duckdb
con = duckdb.connect('md:')

In [4]:
con.sql("""SELECT 
    DATE_TRUNC('week', download_date) AS week_start_date,
    version,
    country_code,
    python_version,
    SUM(daily_download_sum) AS weekly_download_sum 
FROM 
   duckdb_stats.main.pypi_daily_stats 
GROUP BY 
    ALL
ORDER BY 
    week_start_date""")

## Friendly SQL

In [7]:
-- function chaining
SELECT
     ('Make it so')
          .UPPER()
          .string_split(' ')
          .list_aggr('string_agg','.')
          .concat('.') AS im_not_messing_around_number_one

,im_not_messing_around_number_one
0,MAKE.IT.SO.


In [8]:
-- exclude and replace
FROM sample_data.who.ambient_air_quality
SELECT * EXCLUDE (who_region, iso3)

,country_name,city,year,version,pm10_concentration,pm25_concentration,no2_concentration,pm10_tempcov,pm25_tempcov,no2_tempcov,type_of_stations,reference,web_link,population,population_source,latitude,longitude,who_ms
0,India,Chennai,2018,version 2022,NaN,30.0,NaN,NaN,91.0,NaN,NA,"U.S. Department of State, United States Enviro...",https://www.airnow.gov/index.cfm?action=airnow...,9890427,NA,13.087840,80.278473,1
1,India,Solapur,2016,"version 2022, version 2018",NaN,39.0,NaN,NaN,99.0,NaN,NA,"Central Pollution Control Board India, Environ...",NA,985568,NA,17.659920,75.906387,1
2,India,Chennai,2019,version 2022,NaN,39.0,NaN,NaN,85.0,NaN,NA,"U.S. Department of State, United States Enviro...","[[[""EPA AirNow DOS"",""http://airnow.gov/index.c...",9890427,NA,13.087840,80.278473,1
3,India,Hyderabad,2019,version 2022,NaN,42.0,NaN,NaN,87.0,NaN,NA,"U.S. Department of State, United States Enviro...","[[[""EPA AirNow DOS"",""http://airnow.gov/index.c...",8943523,NA,17.384050,78.456360,1
4,India,Pune,2017,version 2022,NaN,43.0,NaN,NaN,NaN,NaN,NA,"Central Pollution Control Board India, Environ...",http://www.cpcb.gov.in/CAAQM/,5727530,NA,18.505320,73.823837,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41359,Saudi Arabia,Jizan,2014,version 2023,148.0,NaN,NaN,NaN,NaN,NaN,NA,"Ministry of Environment, Water, and Agriculture",NA,127743,NA,16.885876,42.573387,1
41360,Saudi Arabia,Jizan,2013,version 2023,208.0,NaN,NaN,NaN,NaN,NaN,NA,"Ministry of Environment, Water, and Agriculture",NA,127743,NA,16.885876,42.573387,1
41361,Saudi Arabia,Jizan,2012,version 2023,184.0,NaN,NaN,NaN,NaN,NaN,NA,"Ministry of Environment, Water, and Agriculture",NA,127743,NA,16.885876,42.573387,1
41362,Saudi Arabia,Jizan,2011,version 2023,316.0,NaN,NaN,NaN,NaN,NaN,NA,"Ministry of Environment, Water, and Agriculture",NA,127743,NA,16.885876,42.573387,1


In [9]:
-- dynamic column selection
FROM sample_data.who.ambient_air_quality
SELECT COLUMNS(c -> c ILIKE 'pop%')

,population,population_source
0,9890427,NA
1,985568,NA
2,9890427,NA
3,8943523,NA
4,5727530,NA
...,...,...
41359,127743,NA
41360,127743,NA
41361,127743,NA
41362,127743,NA


In [14]:
-- group by all
FROM sample_data.hn.hacker_news
SELECT
    YEAR(timestamp) AS year,
    MONTH(timestamp) AS month,
    COUNT(*) AS keyword_mentions

WHERE
    (title LIKE '%duckdb%' OR text LIKE '%duckdb%')
GROUP BY year, month
ORDER BY year ASC, month ASC;

,year,month,keyword_mentions
0,2022,1,6
1,2022,2,4
...,...,...,...
9,2022,10,10
10,2022,11,9


In [15]:
-- COLUMN()
SELECT
    COLUMNS('.*pop.*')
FROM sample_data.who.ambient_air_quality

,population,population_source
0,9890427,NA
1,985568,NA
...,...,...
41362,127743,NA
41363,127743,NA
